# Pivot Tables & Reshaping

Vamos a ver algunas técnicas para manejar DataFrames

In [65]:
import pandas as pd
import numpy as np

In [66]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [67]:
df.head(10)

,date,variable,value
0,2000-01-03,A,0.585463
1,2000-01-04,A,-0.225390
2,2000-01-05,A,0.187171
3,2000-01-03,B,0.203887
4,2000-01-04,B,-1.532304
5,2000-01-05,B,0.172215
6,2000-01-03,C,-1.123566
7,2000-01-04,C,-0.329053
8,2000-01-05,C,-0.455719
9,2000-01-03,D,-1.261929


In [68]:
df.pivot(index='date',columns='variable',values='value')

variable,A,B,C,D
date,,,,
2000-01-03,0.585463,0.203887,-1.123566,-1.261929
2000-01-04,-0.225390,-1.532304,-0.329053,-0.398633
2000-01-05,0.187171,0.172215,-0.455719,-0.563963


Si tenemos más de un valor y omitimos pasar el parámetro 'values', entonces pandas creara una serie de columnas jerárquicas por nosotros. Ejemplo:

Creamos una columna adicional 'value2'

In [69]:
df['value2']= df['value']*2

In [70]:
df.head()

,date,variable,value,value2
0,2000-01-03,A,0.585463,1.170926
1,2000-01-04,A,-0.225390,-0.450780
2,2000-01-05,A,0.187171,0.374343
3,2000-01-03,B,0.203887,0.407775
4,2000-01-04,B,-1.532304,-3.064608


In [71]:
pivoted = df.pivot('date','variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.585463  0.203887 -1.123566 -1.261929  1.170926  0.407775   
2000-01-04 -0.225390 -1.532304 -0.329053 -0.398633 -0.450780 -3.064608   
2000-01-05  0.187171  0.172215 -0.455719 -0.563963  0.374343  0.344430   

                                
variable           C         D  
date                            
2000-01-03 -2.247131 -2.523858  
2000-01-04 -0.658105 -0.797265  
2000-01-05 -0.911438 -1.127925

In [72]:
type(pivoted)

pandas.core.frame.DataFrame

A la que podemos accesar escribiendo por ejemplo:

In [73]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,1.170926,0.407775,-2.247131,-2.523858
2000-01-04,-0.450780,-3.064608,-0.658105,-0.797265
2000-01-05,0.374343,0.344430,-0.911438,-1.127925


## Reshaping by stacking and unstacking

In [74]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                         'foo', 'foo', 'qux', 'qux'],
                        ['one', 'two', 'one', 'two',
                         'one', 'two', 'one', 'two']]))

In [75]:
?zip

In [76]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [77]:
index = pd.MultiIndex.from_tuples(tuples,names=['first','second'])
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [78]:
df2 = pd.DataFrame(np.random.randn(8,2),index=index, columns=['A','B'])
df2

A         B
first second                    
bar   one    -0.514680  0.431980
      two     0.928087 -1.081042
baz   one     0.402941 -0.530293
      two    -1.017023 -1.657740
foo   one     0.230068  1.270548
      two     0.063703 -0.271330
qux   one     1.943334  0.627181
      two     0.496295  1.381683

La función **stack** comprime un nivel las columnas del DataFrame para producir ya sea:
<li> Una Serie, en el caso de un índice de una sóla columna </li>
<li> Un DataFrame, en el caso de columnas con múltiples índices </li>

In [79]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.514680
               B    0.431980
       two     A    0.928087
               B   -1.081042
baz    one     A    0.402941
               B   -0.530293
       two     A   -1.017023
               B   -1.657740
foo    one     A    0.230068
               B    1.270548
       two     A    0.063703
               B   -0.271330
qux    one     A    1.943334
               B    0.627181
       two     A    0.496295
               B    1.381683
dtype: float64

La operación inversa de stack es **unstack** que por default descomprime el último nivel

In [80]:
stacked.unstack()

A         B
first second                    
bar   one    -0.514680  0.431980
      two     0.928087 -1.081042
baz   one     0.402941 -0.530293
      two    -1.017023 -1.657740
foo   one     0.230068  1.270548
      two     0.063703 -0.271330
qux   one     1.943334  0.627181
      two     0.496295  1.381683

In [81]:
stacked.unstack(0)

first          bar       baz       foo       qux
second                                          
one    A -0.514680  0.402941  0.230068  1.943334
       B  0.431980 -0.530293  1.270548  0.627181
two    A  0.928087 -1.017023  0.063703  0.496295
       B -1.081042 -1.657740 -0.271330  1.381683

In [82]:
stacked.unstack(1)

second        one       two
first                      
bar   A -0.514680  0.928087
      B  0.431980 -1.081042
baz   A  0.402941 -1.017023
      B -0.530293 -1.657740
foo   A  0.230068  0.063703
      B  1.270548 -0.271330
qux   A  1.943334  0.496295
      B  0.627181  1.381683

Si los niveles tienen nombres como es el caso, entonces podemos referirnos a ellos por nombre

In [83]:
stacked.unstack('second')

second        one       two
first                      
bar   A -0.514680  0.928087
      B  0.431980 -1.081042
baz   A  0.402941 -1.017023
      B -0.530293 -1.657740
foo   A  0.230068  0.063703
      B  1.270548 -0.271330
qux   A  1.943334  0.496295
      B  0.627181  1.381683

Nota que los métodos stack y unstack implícitamente ordenan los levels por índice por lo que aplicar unstack y stack nos regresas un nuevo DataFrame ordenado

Generemos un nuevo DataFrame para trabajar y probar nuestra teoría

In [84]:
index = pd.MultiIndex.from_product([[2,1], ['a', 'b']])

In [85]:
df = pd.DataFrame(np.random.randn(4), index=index, columns=['A'])
df

A
2 a  0.867635
  b  1.669287
1 a  0.453452
  b -0.708299

In [86]:
all(df.unstack().stack() == df.sort_index())

True

Comprobada! 

### Trabajando con mútiples levels (niveles)

Primero generemos el DataFrame

In [87]:
columns = pd.MultiIndex.from_tuples([
            ('A', 'cat', 'long'), ('B', 'cat', 'long'),
            ('A', 'dog', 'short'), ('B', 'dog', 'short')
        ],
        names=['exp', 'animal', 'hair_length']
    )

In [88]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-0.586704,-1.113524,0.505546,-0.785500
1,-1.618846,1.102545,0.071818,0.146051
2,-1.830518,-0.436956,0.011143,0.671305
3,-0.187962,1.288162,-0.057921,1.210106


La lista de niveles puede tener sin problema el nombre de 'levels' (los niveles) o el índice de los niveles. Entonces

In [89]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.586704 -1.113524
  dog    short        0.505546 -0.785500
1 cat    long        -1.618846  1.102545
  dog    short        0.071818  0.146051
2 cat    long        -1.830518 -0.436956
  dog    short        0.011143  0.671305
3 cat    long        -0.187962  1.288162
  dog    short       -0.057921  1.210106

alternativamente

In [90]:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.586704 -1.113524
  dog    short        0.505546 -0.785500
1 cat    long        -1.618846  1.102545
  dog    short        0.071818  0.146051
2 cat    long        -1.830518 -0.436956
  dog    short        0.011143  0.671305
3 cat    long        -0.187962  1.288162
  dog    short       -0.057921  1.210106

### Valores Faltantes (Missing Values)

Generemos primero el DataFrame

In [91]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                         ('B', 'cat'), ('A', 'dog')],
                                        names=['exp', 'animal'])
    

In [92]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                        ('one', 'two')],
                                       names=['first', 'second'])
    

In [93]:
df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.573444 -1.886642 -0.296534 -0.362406
      two    -2.567233  0.427774  0.605328 -0.592754
baz   one    -0.055543 -0.042737 -0.232439  0.633023
      two     0.403394 -0.203807 -1.607800 -0.565935
foo   one    -1.992386  0.098007 -1.053219 -1.724144
      two    -1.609317 -1.172114 -0.286725 -0.061338
qux   one    -0.268931 -0.051576  0.532569 -1.133503
      two    -0.071953 -1.514956 -0.513448  0.931387

In [94]:
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.573444 -1.886642 -0.296534 -0.362406
      two    -2.567233  0.427774  0.605328 -0.592754
baz   one    -0.055543 -0.042737 -0.232439  0.633023
foo   one    -1.992386  0.098007 -1.053219 -1.724144
      two    -1.609317 -1.172114 -0.286725 -0.061338
qux   two    -0.071953 -1.514956 -0.513448  0.931387

Entonces podemos seleccionar con el método stack para seleccionar a que nivel queremos las columnas a acomodar

In [95]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A   -0.573444 -0.362406
             B   -0.296534 -1.886642
      two    A   -2.567233 -0.592754
             B    0.605328  0.427774
baz   one    A   -0.055543  0.633023
             B   -0.232439 -0.042737
foo   one    A   -1.992386 -1.724144
             B   -1.053219  0.098007
      two    A   -1.609317 -0.061338
             B   -0.286725 -1.172114
qux   two    A   -0.071953  0.931387
             B   -0.513448 -1.514956

In [96]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat    -0.573444 -0.296534
             dog    -0.362406 -1.886642
      two    cat    -2.567233  0.605328
             dog    -0.592754  0.427774
baz   one    cat    -0.055543 -0.232439
             dog     0.633023 -0.042737
foo   one    cat    -1.992386 -1.053219
             dog    -1.724144  0.098007
      two    cat    -1.609317 -0.286725
             dog    -0.061338 -1.172114
qux   two    cat    -0.071953 -0.513448
             dog     0.931387 -1.514956

Ahora veamos que hacer unstacking puede resultar en valores faltantes si los subgrupos no tienen el mismo conjunto de etiquetas

In [97]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -1.886642 -0.296534
      two     0.427774  0.605328
foo   one     0.098007 -1.053219
qux   two    -1.514956 -0.513448

In [98]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -1.886642  0.427774 -0.296534  0.605328
foo     0.098007       NaN -1.053219       NaN
qux          NaN -1.514956       NaN -0.513448

Para subsanar este problema, unstack toma el argumento 'fill_value' para especificar el tratamiento que le queremos dar a esos valores faltantes (missing values)

In [99]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -1.886642e+00  4.277739e-01 -2.965341e-01  6.053285e-01
foo     9.800737e-02 -1.000000e+09 -1.053219e+00 -1.000000e+09
qux    -1.000000e+09 -1.514956e+00 -1.000000e+09 -5.134480e-01

### Valores faltantes con multi-indexing

In [100]:
df[:3]

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.573444 -1.886642 -0.296534 -0.362406
      two    -2.567233  0.427774  0.605328 -0.592754
baz   one    -0.055543 -0.042737 -0.232439  0.633023

In [35]:
df[:3].unstack(0)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
first        bar       baz       bar       baz       bar       baz       bar   
second                                                                         
one    -0.263316  1.004609  0.265991 -0.196877 -0.421131  0.541896 -1.284754   
two    -1.791886       NaN  1.011910       NaN -0.659064       NaN  0.991658   

exp               
animal            
first        baz  
second            
one    -0.509182  
two          NaN

In [36]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar    -0.263316 -1.791886  0.265991  1.011910 -0.421131 -0.659064 -1.284754   
baz     1.004609       NaN -0.196877       NaN  0.541896       NaN -0.509182   
foo     0.020466 -0.743200  0.795574  0.154623 -0.106643 -0.968228 -1.420256   
qux          NaN  1.373119       NaN -0.621956       NaN  0.670785       NaN   

exp               
animal            
second       two  
first             
bar     0.991658  
baz          NaN  
foo     1.516166  
qux    -0.574300

### Masajeando un DataFrame con melt

In [37]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                           'last' : ['Doe', 'Bo'],
                           'height' : [5.5, 6.0],
                           'weight' : [130, 150]})
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [38]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [39]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


### Combinando un estadística y GroupBy

In [101]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.573444 -1.886642 -0.296534 -0.362406
      two    -2.567233  0.427774  0.605328 -0.592754
baz   one    -0.055543 -0.042737 -0.232439  0.633023
      two     0.403394 -0.203807 -1.607800 -0.565935
foo   one    -1.992386  0.098007 -1.053219 -1.724144
      two    -1.609317 -1.172114 -0.286725 -0.061338
qux   one    -0.268931 -0.051576  0.532569 -1.133503
      two    -0.071953 -1.514956 -0.513448  0.931387

In [102]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat    -0.573444 -0.296534
             dog    -0.362406 -1.886642
      two    cat    -2.567233  0.605328
             dog    -0.592754  0.427774
baz   one    cat    -0.055543 -0.232439
             dog     0.633023 -0.042737
      two    cat     0.403394 -1.607800
             dog    -0.565935 -0.203807
foo   one    cat    -1.992386 -1.053219
             dog    -1.724144  0.098007
      two    cat    -1.609317 -0.286725
             dog    -0.061338 -1.172114
qux   one    cat    -0.268931  0.532569
             dog    -1.133503 -0.051576
      two    cat    -0.071953 -0.513448
             dog     0.931387 -1.514956

In [103]:
df.stack().mean(1)

first  second  animal
bar    one     cat      -0.434989
               dog      -1.124524
       two     cat      -0.980952
               dog      -0.082490
baz    one     cat      -0.143991
               dog       0.295143
       two     cat      -0.602203
               dog      -0.384871
foo    one     cat      -1.522802
               dog      -0.813068
       two     cat      -0.948021
               dog      -0.616726
qux    one     cat       0.131819
               dog      -0.592539
       two     cat      -0.292701
               dog      -0.291784
dtype: float64

In [104]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one    -0.434989 -1.124524
      two    -0.980952 -0.082490
baz   one    -0.143991  0.295143
      two    -0.602203 -0.384871
foo   one    -1.522802 -0.813068
      two    -0.948021 -0.616726
qux   one     0.131819 -0.592539
      two    -0.292701 -0.291784

Alternativamente

In [42]:
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one    -0.342224 -0.509381
      two    -1.225475  1.001784
baz   one     0.773252 -0.353030
      two    -0.771028  2.161350
foo   one    -0.043089 -0.312341
      two    -0.855714  0.835395
qux   one     0.418577  0.298551
      two     1.021952 -0.598128

In [43]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,-0.337322,0.319901
two,0.389773,0.002761


In [44]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.258862,0.002925
dog,0.311313,0.319737


### Por fin.... Tablas dinámicas!!

In [45]:
import datetime
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                       'B': ['A', 'B', 'C'] * 8,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                       'D': np.random.randn(24),
                       'E': np.random.randn(24),
                       'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                            [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df

,A,B,C,D,E,F
0,one,A,foo,-0.835172,1.799015,2013-01-01
1,one,B,foo,1.833729,2.070393,2013-02-01
2,two,C,foo,0.257224,-1.367759,2013-03-01
3,three,A,bar,0.307289,2.030035,2013-04-01
4,one,B,bar,-0.423991,-0.265204,2013-05-01
5,one,C,bar,0.810630,1.739166,2013-06-01
6,two,A,foo,0.763770,0.974195,2013-07-01
7,three,B,foo,0.296675,-0.574531,2013-08-01
8,one,C,foo,-0.414770,-0.590020,2013-09-01
9,one,A,bar,0.849094,0.350371,2013-10-01


Podemos pedirle que nos haga una tabla dinámica de esta información de la siguiente forma

In [46]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.537120 -1.230993
      B -1.142890  0.917770
      C  0.089520  0.662680
three A  0.941718       NaN
      B       NaN -0.979393
      C  0.948707       NaN
two   A       NaN  0.047028
      B -0.314629       NaN
      C       NaN -0.429362

In [47]:
pd.pivot_table(df, values='D', index=['B'], columns=['A', 'C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  1.074239 -2.461986  1.883436       NaN       NaN  0.094057
B -2.285780  1.835541       NaN -1.958787 -0.629257       NaN
C  0.179040  1.325360  1.897414       NaN       NaN -0.858725

In [48]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  1.074239 -2.461986  1.883436       NaN       NaN  0.094057 -0.356630   
B -2.285780  1.835541       NaN -1.958787 -0.629257       NaN -0.426053   
C  0.179040  1.325360  1.897414       NaN       NaN -0.858725  3.441882   

                                                    
A               three                two            
C       foo       bar      foo       bar       foo  
B                                                   
A  0.855697  1.771187      NaN       NaN -0.064347  
B  1.902528       NaN -0.64368 -2.309317       NaN  
C -0.779012 -2.756801      NaN       NaN -1.933443

El objeto resultante es un DataFrame con los índices jerárquicos en filas y columnas. Si no le pasamos el argumento 'values' entonces los datos podrían agregarse en los niveles jerárquicos superiores como por ejemplo:

In [49]:
pd.pivot_table(df, index=['A', 'B'], columns=['C'])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.537120 -1.230993 -0.178315  0.427849
      B -1.142890  0.917770 -0.213027  0.951264
      C  0.089520  0.662680  1.720941 -0.389506
three A  0.941718       NaN  0.885594       NaN
      B       NaN -0.979393       NaN -0.321840
      C  0.948707       NaN -1.378401       NaN
two   A       NaN  0.047028       NaN -0.032174
      B -0.314629       NaN -1.154659       NaN
      C       NaN -0.429362       NaN -0.966722

Para mejorar el output de una pivot table se puede usar el método .to_string() de la siguiente forma

In [50]:
table = pd.pivot_table(df, index=['A', 'B'], columns=['C'])

In [51]:
print(table.to_string(na_rep=''))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.537120 -1.230993 -0.178315  0.427849
      B -1.142890  0.917770 -0.213027  0.951264
      C  0.089520  0.662680  1.720941 -0.389506
three A  0.941718            0.885594          
      B           -0.979393           -0.321840
      C  0.948707           -1.378401          
two   A            0.047028           -0.032174
      B -0.314629           -1.154659          
      C           -0.429362           -0.966722


Si quieres la agregación de filas y columnas en los márgenes de la tabla puedes adicionarlos sin problema usando el argumento 'margins=True' de la siguiente forma:

In [52]:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.441199  0.559776  1.100640  0.747674  1.939123  1.249886
      B  1.016677  1.295361  1.522926  0.073791  1.582687  1.135182
      C  1.019803  1.523745  1.109100  0.025774  0.283570  1.229507
three A  0.897218       NaN  0.897218  1.618485       NaN  1.618485
      B       NaN  1.804634  1.804634       NaN  0.357359  0.357359
      C  0.650295       NaN  0.650295  0.604312       NaN  0.604312
two   A       NaN  1.013625  1.013625       NaN  1.423220  1.423220
      B  1.965972       NaN  1.965972  1.164925       NaN  1.164925
      C       NaN  0.970980  0.970980       NaN  0.567152  0.567152
All      1.126707  1.246217  1.150420  1.313767  1.101303  1.160593